# Analysis of the results

This notebook investigates the results of running this analysis.

## Imports and hardcoded variables

In [ ]:
import json
import os
from pathlib import Path

import arviz as az
import numpy as np
import pandas as pd
import scipy
import xarray

from matplotlib import pyplot as plt
from pprint import pprint

from equilibrator_api import ComponentContribution
from dgfreg.data_preparation import load_prepared_data

INFERENCES_DIR = os.path.join("..", "inferences")
PLOTS_DIR = os.path.join("..", "docs", "plots")
ARVIZ_STYLE = "arviz-redish"

plt.style.use(ARVIZ_STYLE)

## Loading InferenceData objects

The results of the analysis are stored as [`InferenceData`](https://arviz-devs.github.io/arviz/api/generated/arviz.InferenceData.html#arviz.InferenceData) objects in json files. The next cell loads these files.

In [ ]:
idata_c = az.from_json(os.path.join(INFERENCES_DIR, "equilibrator_component", "idata.json"))
idata_c

## Extracting draws

In [ ]:
# idata_c = idatas["equilibrator_component"]

idata_c.observed_data.coords["observation"] = idata_c.posterior_predictive.coords["observation"]
idata_c.observed_data["y"] = idata_c.observed_data["y"].rename({"y_dim_0": "observation"})

dgfC_draws = idata_c.posterior["dgfC"]
dgfG_draws = idata_c.posterior["dgfG"]
dgr_draws = idata_c.posterior["dgr"]

dgfC_qs = dgfC_draws.quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("q")
dgfC_qs.index = dgfC_qs.index.astype(int)

dgr_qs = dgr_draws.quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("q")
dgr_qs.index = dgr_qs.index.astype(int)

yrep_qs = idata_c.posterior_predictive["yrep"].quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("yrepq")
yrep_qs.index = dgr_qs.index.astype(int)

resids = idata_c.posterior_predictive["yrep"] - idata_c.observed_data["y"]
resid_qs = resids.quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("resid_q")
resid_qs.index = resid_qs.index.astype(int)
resid_qs = resid_qs.rename_axis("reaction_id")
resid_mean = idata_c.posterior_predictive["yrep"].mean(dim=["chain", "draw"]) - idata_c.observed_data["y"]

az.summary(idata_c, var_names=["tauC", "sigma", "mae"])

## Gathering component contribution information

In [ ]:
cc = ComponentContribution()

In [ ]:
cc_cov = pd.read_csv(Path.cwd().parent / "data" / "raw" / "dgf_cov.csv").rename(columns={"Unnamed: 0": "compound_id"}).set_index("compound_id")
cc_cov.columns = map(int, cc_cov.columns)
sd_cc = pd.Series(np.sqrt(np.diag(cc_cov)), index=cc_cov.index).rename("sd_cc")

dgr_gc = (cc.predictor.params.train_G.T @ cc.predictor.params.train_S).T @ cc.predictor.params.dG0_gc
dgr_cc = cc.predictor.params.train_S.T @ cc.predictor.params.dG0_cc

In [ ]:
prepped = load_prepared_data(Path.cwd().parent / "data" / "prepared" / "equilibrator")

measurements = prepped.measurements.set_index("measurement_id")
measurements["dgr_gc"] = dgr_gc
measurements["dgr_cc"] = dgr_cc
measurements["resid_cc"] = dgr_cc - measurements["y"]
measurements["resid_gc"] = dgr_gc - measurements["y"]

G = prepped.G.set_index(["compound_id", "group_id"])["stoichiometric_coefficient"].unstack().fillna(0.0)
S = prepped.S.set_index(["compound_id", "reaction_id"])["stoichiometric_coefficient"].unstack().fillna(0.0)

reactions = prepped.reactions.set_index("reaction_id")
reactions["conserves_groups"] = (G.T @ S).abs().sum() == 0
if "q0.5" not in reactions.columns:
    reactions = reactions.join(dgr_qs)
if "resid_q0.5" not in reactions.columns:
    reactions = reactions.join(resid_qs)
reactions["resid_mean"] = resid_mean
reactions["dgr_cc"] = measurements.groupby("reaction_id")["dgr_cc"].first()
reactions["dgr_gc"] = measurements.groupby("reaction_id")["dgr_gc"].first()
reactions = reactions.join(measurements.groupby("reaction_id").agg({"y": ["mean", "count"]})["y"].add_prefix("y_"))

if "q0.5" not in measurements.columns:
    measurements = measurements.join(resid_qs, on="reaction_id")

compounds = prepped.compounds.set_index("compound_id")
compounds["sd_cc"] = sd_cc
compounds["q0.01_cc"], compounds["q0.99_cc"] = compounds["dgf_cc"] - 2.33 * sd_cc, compounds["dgf_cc"] + 2.33 * sd_cc
if "q0.5" not in compounds.columns:
    compounds = compounds.join(dgfC_qs)
compounds.head()

In [ ]:
cc_cov_long = (
    cc_cov.mask(np.triu(np.ones(cc_cov.shape)).astype(bool))
    .stack()
    .sort_values()
    .reset_index()
    .rename(columns={"compound_id": "compound_id_1", "level_1": "compound_id_2", 0: "covariance"})
    .assign(compound_id_2=lambda df: df["compound_id_2"].astype(int))
)
core_mets = compounds.loc[lambda df: df["is_e_coli_ccm"]].index
(
    cc_cov_long
    .loc[lambda df: df["compound_id_1"].isin(core_mets)]
    .tail(20)
    .join(compounds[["common_name", "is_e_coli_ccm"]], on="compound_id_1")
    .join(compounds[["common_name", "is_e_coli_ccm"]], on="compound_id_2", lsuffix="_1", rsuffix="_2")
)



## Add model information to measurements dataframe

In [ ]:
if "q0.01" not in measurements.columns:
    measurements = measurements.join(dgr_qs, on="reaction_id")
if "yrepq0.01" not in measurements.columns:
    measurements = measurements.join(yrep_qs, on="reaction_id")
if "resid_mean" in measurements.columns:
    measurements = measurements.drop("resid_mean", axis=1)
measurements = measurements.join(reactions["resid_mean"], on="reaction_id")

In [ ]:
rmse_posterior = measurements[["resid_mean", "resid_cc"]].pow(2).mean().pipe(np.sqrt)

f, ax = plt.subplots(figsize=[8, 5])
bins = np.linspace(-25, 25, 50)
ax.hist(
    measurements["resid_mean"], 
    bins=bins, 
    alpha=0.7,
    density=True, 
    label=f"Posterior predictive mean\nRMSE {rmse_posterior['resid_mean'].round(1)}"
);
ax.hist(
    measurements["resid_cc"], 
    color="orange", bins=bins, 
    alpha=0.7, 
    density=True, 
    label=f"Component contribution estimate\nRMSE {rmse_posterior['resid_cc'].round(1)}"
);
ax.set(
    title="Our model fits the TECRDB data about as well as component contribution",
    ylabel="Relative frequency in training measurements",
    xlabel="$\Delta_rG$ residual (kJ/mol)"
);
ax.legend(frameon=False);
f.savefig(os.path.join(PLOTS_DIR, "in_sample_rmse_comparison.svg"), bbox_inches="tight");


In [ ]:
grouped = measurements.groupby("reaction_id")

measurements_by_reaction = (
    reactions[["is_e_coli_ccm", "description", "q0.01", "q0.5", "q0.99", "dgr_cc"]]
    .join(grouped.agg({"y": ["mean", "count"]})["y"].add_prefix("y_"))
    .assign(
        resid_low=lambda df: df["q0.01"] - df["y_mean"],
        resid_high=lambda df: df["q0.99"] - df["y_mean"],
        resid_med=lambda df: df["q0.5"] - df["y_mean"],
        resid_cc=lambda df: df["dgr_cc"] - df["y_mean"]
    )
)

f, axes = plt.subplots(2, 1, figsize=[15, 10])

for ccm_only, ax in zip([False, True], axes):
    plot_df = measurements_by_reaction.loc[lambda df: df["is_e_coli_ccm"]] if ccm_only else measurements_by_reaction
    plot_df = plot_df.sort_values("resid_med")
    x = np.linspace(*ax.get_xlim(), len(plot_df))
    vlines = ax.vlines(x, plot_df["resid_low"], plot_df["resid_high"], label="1%-99% posterior predictive interval");
    hline = ax.axhline(0, color="black", linestyle="--")
    scatter = ax.scatter(x, plot_df["resid_cc"], color="orange", zorder=2, marker="x", label="Component contribution");
    ax.set_xticks([])
    ax.set(
        title="E coli core reactions" if ccm_only else "All reactions",
        ylabel="Residual"
    )
    if ccm_only:
        for i, (ix, row) in enumerate(plot_df.iterrows()):
            if row["resid_low"] > 2 or row["resid_high"] < -1:
                ax.text(x[i], row["resid_low"], row["description"], fontsize=8)
f.legend(
    [vlines, scatter],
    ["1%-99% posterior predictive interval", "Component contribution estimate"], 
    frameon=False, 
    loc="upper center",
    bbox_to_anchor=[0.5, 0.95],
    ncol=2,
);
f.suptitle("Posterior predictive check", fontsize=16);

f.savefig(os.path.join(PLOTS_DIR, "marginal_ppc.svg"), bbox_inches="tight");


In [ ]:
f, ax = plt.subplots(figsize=[15, 5])
az.plot_ppc(idata_c, data_pairs={"y": "yrep"}, var_names=["y"], ax=ax);
ax.set_xlim(-100, 150);
f.savefig(os.path.join(PLOTS_DIR, "ppc_kde.svg"), bbox_inches="tight");

## Plots of interesting parameters

In [ ]:
vars = ["tauC", "sigma"]
priors = {"tauC": [0, 4], "sigma": [0, 4]}

az.plot_posterior(idata_c, var_names=vars, kind="hist", label="posterior draws", hdi_prob="hide");

f = plt.gcf()
axes = f.get_axes()
for ax, var in zip(axes, vars):
    loc, scale = priors[var]
    dist = scipy.stats.norm(loc=loc, scale=scale).pdf(x)
    x = np.linspace(*ax.get_xlim(), 50)
    ax.legend(frameon=False)

f.savefig(os.path.join(PLOTS_DIR, "posteriors.svg"), bbox_inches="tight");

## Gaussian approximation of posterior distribution

These tables can be used if you would like a Gaussian approximation to the $\Delta_fG$ posterior distributions from our model.

In [ ]:
mean = dgfC_draws.mean(dim=["chain", "draw"]).to_series().rename("mean")
cov = pd.DataFrame(np.cov(dgfC_draws.to_dataframe().unstack(["chain", "draw"])), index=compounds.index, columns=compounds.index)

mean.to_csv(os.path.join(INFERENCES_DIR, "gaussian_approximation_mean.csv"))
cov.to_csv(os.path.join(INFERENCES_DIR, "gaussian_approximation_cov.csv"))

ccm = compounds.loc[lambda df: df["is_e_coli_ccm"]].index.values

display(mean)
display(cov)

In [ ]:
print(f"Shape of formation energy covariance matrix: {cov.shape}")
print(f"Rank of component contribution formation energy covariance matrix: {np.linalg.matrix_rank(cc_cov)}")
print(f"Rank of new model formation energy covariance matrix: {np.linalg.matrix_rank(cov)}")

## Comparison with component contribution

First I wanted to find how many *E. coli* core reactions conserved chemical groups.

In [ ]:
bad = reactions.loc[lambda df: df["conserves_groups"], ["description", "EC"]]

bad_e_coli_core =  reactions.loc[lambda df: df["conserves_groups"] & df["is_e_coli_ccm"], ["description", "EC"]]

display(len(bad))
display(bad_e_coli_core)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=[10, 8])
for ccm_only, ax in zip([True, False], axes):
    plot_df = compounds if not ccm_only else compounds.loc[lambda df: df["is_e_coli_ccm"]]
    plot_df = plot_df.sort_values("sd_cc")
    y = np.linspace(*ax.get_ylim(), len(plot_df))
    model_lines = ax.hlines(
        y,
        plot_df["q0.01"] - plot_df["dgf_cc"],
        plot_df["q0.99"] - plot_df["dgf_cc"],
        alpha=0.8,
    )
    cc_lines = ax.hlines(
        y, 
        plot_df["q0.01_cc"] - plot_df["dgf_cc"], 
        plot_df["q0.99_cc"] - plot_df["dgf_cc"],
        color="orange", 
        alpha=0.8,
    )
    equal_line = ax.axvline(0, color="black")
    if ccm_only:
        ticks = axes[0].set_yticks(y)
        txt = axes[0].set_yticklabels(plot_df["common_name"])
    ax.set(xlabel="Difference from cc estimate (kJ/mol)", title="E coli core compounds" if ccm_only else "All compounds")
leg = f.legend(
    [model_lines, cc_lines, equal_line],
    ["Hierarchical model", "Component contribution", "Component contribution estimate"],
    ncol=3,
    frameon=False,
    loc="lower center"
)
suptitle = f.suptitle("1%-99% marginal formation energy uncertainty intervals", fontsize=16, y=0.95)

f.savefig(os.path.join(PLOTS_DIR, "marginal_dgf_comparison.svg"), bbox_inches="tight");

In [ ]:
f, ax = plt.subplots()

ax.vlines(compounds["dgf_cc"], compounds["q0.01"], compounds["q0.99"], label="1%-99% posterior interval");
ax.plot([*ax.get_xlim()], [*ax.get_xlim()], color="black", label="y=x");
ax.set(
    title="Overall agreement between formation energy estimates",
    xlabel="CC estimate (kJ/mol)",
    ylabel="New model estimate (kJ/mol)"
)
ax.legend(frameon=False);
f.savefig(os.path.join(PLOTS_DIR, "dgf_comparison_scatter.svg"), bbox_inches="tight");

In [ ]:
widths_model = compounds["q0.99"] - compounds["q0.01"]
widths_cc = compounds["q0.99_cc"] - compounds["q0.01_cc"]
med_model = widths_model.mean()
med_cc = widths_cc.mean()

f, ax = plt.subplots(figsize=[7, 5])
bins = np.linspace(0, 200, 50)
ax.set(
    title="Distribution of formation energy 1%-99% uncertainty quantile widths",
    ylabel="Relative frequency",
    xlabel="1%-99% marginal uncertainty quantile width (kJ/mol)"
)
ax.hist(widths_model, density=True, bins=bins, alpha=0.8, label="model");
ax.hist(widths_cc, color="orange", density=True, bins=bins, alpha=0.8, label="Component Contribution");
ax.axvline(med_model, label="mean (model)")
ax.axvline(med_cc, color="orange", label="mean (Component Contribution)")
ax.legend(frameon=False);
f.savefig(os.path.join(PLOTS_DIR, "marginal_width_hists.svg"), bbox_inches="tight");

# Out of sample case study

To see how the new model compares with component contribution when modelling unseen compounds, we constructed a custom test dataset.

The test dataset removes from the standard training dataset all reactions involving the compounds D-Glucose 6-phosphate, Phosphoenolpyruvate and Acetyl-CoA.

In [ ]:
from dgfreg.component_contribution_replication import fit_component_contribution
excluded_compounds = [23, 70, 152]
excluded_reactions = (
    S
    .loc[excluded_compounds]
    .replace(0, np.nan)
    .stack()
    .reset_index()
    ["reaction_id"]
    .unique()
)
test_y = measurements.loc[lambda df: ~df["reaction_id"].isin(excluded_reactions)]
test_S = cc.predictor.params.train_S[test_y.index]
cc_test_fit = fit_component_contribution(test_S, G, test_y)

if "dgr_cc_test" in measurements.columns:
    measurements = measurements.drop("dgr_cc_test", axis=1)
measurements = measurements.join((S.T @ cc_test_fit.mu_dgf).rename("dgr_cc_test"), on="reaction_id")
measurements["resid_cc_test"] = measurements["dgr_cc_test"] - measurements["y"]

if "dgr_cc_test" in reactions.columns:
    reactions = reactions.drop("dgr_cc_test", axis=1)
reactions = reactions.join((S.T @ cc_test_fit.mu_dgf).rename("dgr_cc_test"), on="reaction_id")
reactions["resid_cc_test"] = reactions["dgr_cc_test"] - reactions["y_mean"]

In [ ]:
test_y.shape, measurements.shape

In [ ]:
idata_test = az.from_json(os.path.join(INFERENCES_DIR, "equilibrator_custom_test", "idata.json"))
idata_test.posterior_predictive["yrep"] = idata_test.posterior_predictive["yrep"].rename({"observation": "reaction_id"})
idata_test.posterior_predictive.coords["reaction_id"] = idata_test.posterior_predictive.coords["reaction_id"] + 1

az.summary(idata_test, var_names=["tauC", "sigma", "mae"])

In [ ]:
yrep_qs_test = idata_test.posterior_predictive["yrep"].quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("yrep_q").add_suffix("_test")
yrep_qs_test.index = yrep_qs_test.index.astype(int)
yrep_mean_test = idata_test.posterior_predictive["yrep"].mean(dim=["chain", "draw"])

resids_test = idata_test.posterior_predictive["yrep"] - reactions["y_mean"].values
resid_qs_test = resids_test.quantile([0.01, 0.5, 0.99], dim=["chain", "draw"]).to_series().unstack("quantile").add_prefix("resid_q").add_suffix("_test")
resid_qs_test.index = resid_qs_test.index.astype(int)
resid_qs_test = resid_qs_test.rename_axis("reaction_id")

if "yrep_q0.5_test" not in reactions.columns:
    reactions = reactions.join(yrep_qs_test)
if "resid_q0.5_test" not in reactions.columns:
    reactions = reactions.join(resid_qs_test)
reactions["yrep_mean_test"] = yrep_mean_test
reactions["resid_yrep_mean_test"] = reactions["yrep_mean_test"] - reactions["y_mean"]

if "yrep_mean_test" not in measurements.columns:
    measurements = measurements.join(reactions["yrep_mean_test"], on="reaction_id")
measurements["resid_yrep_mean_test"] = measurements["y"] - measurements["yrep_mean_test"]
rmse_test = measurements[["resid_cc_test", "resid_yrep_mean_test"]].drop(test_y.index).pow(2).mean().pipe(np.sqrt)
rmse_test

In [ ]:
test_resid_diffs = (
    measurements[["resid_cc_test", "resid_yrep_mean_test"]]
    .drop(test_y.index)
    .abs()
    .diff(axis=1)
    ["resid_yrep_mean_test"]
)
outlier = test_resid_diffs.sort_values().iloc[0]

f, ax = plt.subplots()
bins = np.linspace(-20, 20, 23)
ax.hist(
    test_resid_diffs,
    bins=bins,
    label=""
);
ax.annotate(
    f"One outlier not shown\n(value {round(outlier, 1)})", 
    xy=[-22, 30],
    xytext=[-19, 25],
    arrowprops={"arrowstyle": "->"},
    fontsize="small"
);
ax.text(-20, 175, "Measurements where our model\nmade a better prediction", fontsize="small")
ax.text(2, 175, "Measurements where component\ncontribution made a better prediction", fontsize="small")
ax.axvline(0, color="black", linestyle="--")
ax.set(
    title="Distribution of absolute residual differences for test measurements\n(our model - component contribution)",
    ylabel="Frequency",
    xlabel="Difference in absolute residuals (kJ/mol)"
);
f.savefig(os.path.join(PLOTS_DIR, "out_of_sample_residual_hist.svg"), bbox_inches="tight");

In [ ]:
dgfC_draws_test = idata_test.posterior["dgfC"]
cov_test = pd.DataFrame(np.cov(dgfC_draws_test.to_dataframe().unstack(["chain", "draw"])), index=compounds.index, columns=compounds.index)

print(f"Shape of formation energy covariance matrix: {cov_test.shape}")
print(f"Rank of new model formation energy covariance matrix when fit to test dataset: {np.linalg.matrix_rank(cov_test)}")
print(f"Rank of component contribution formation energy covariance matrix when fit to test dataset: {np.linalg.matrix_rank(cc_test_fit.cov_dgf)}")


In [ ]:
plot_df = reactions.loc[excluded_reactions].sort_values("resid_q0.5_test").iloc[:-1]
rmse_cc, rmse_new = rmse_test

f, ax = plt.subplots(figsize=[6, 8])
y = np.linspace(*ax.get_ylim(), len(plot_df))


# lines = ax.hlines(y-0.005, test_resid_qs_measurement.loc[ix, "q0.01"], test_resid_qs_measurement.loc[ix, "q0.99"])
red_lines = ax.hlines(y+0.005, plot_df["resid_q0.01_test"], plot_df["resid_q0.99_test"], alpha=0.8)
scatter_red = ax.scatter(plot_df["resid_yrep_mean_test"], y+0.005, zorder=3)

scatter_orange = ax.scatter(plot_df["resid_cc_test"], y, color="orange", marker="x", zorder=3)

ax.axvline(0, linestyle="--", color="black")
ax.set_yticks(y);
ax.set_yticklabels(plot_df["description"], fontsize=8);
# ax.set_xlabel("1%-99% posterior predictive interval: $\Delta_rG$ residual (kJ/mol)")
f.legend(
    [red_lines, scatter_red, scatter_orange],
    ["1%-99% posterior predictive interval", "Posterior mean", "Component contribution"],
    frameon=False, 
    # ncol=3, 
    bbox_to_anchor=[0.12, 0.885],
    fontsize="x-small",
    loc="upper left"
)
ax.set_xlabel("$\Delta_rG$ residual (kJ/mol)")
ax.text(-42, 0.9, "(RMSE figures include an\noutlier reaction that isn't\nshown in this figure)", fontsize="x-small")
f.suptitle("Predictive performance on reactions excluded from the training dataset", x=-0.1, y=0.92);

f.savefig(os.path.join(PLOTS_DIR, "out_of_sample_reaction_residuals.svg"), bbox_inches="tight");
